In [198]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier  # Tambahkan import ini

import optuna
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chocolate-bar-ratings/flavors_of_cacao.csv


In [199]:
# Add this at the start of your Kaggle notebook
!pip install -U scikit-learn==1.2.2
!pip install -U imbalanced-learn==0.10.1

In [200]:
import matplotlib.pyplot as plt
import missingno as mso
import seaborn as sns
import warnings
import scipy

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

from scipy import stats
from scipy.stats import pearsonr
from scipy.stats import ttest_ind
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

In [201]:
data_path = '/kaggle/input/chocolate-bar-ratings/flavors_of_cacao.csv'
cc = pd.read_csv(data_path)
cc

,Company \n(Maker-if known),Specific Bean Origin\nor Bar Name,REF,Review\nDate,Cocoa\nPercent,Company\nLocation,Rating,Bean\nType,Broad Bean\nOrigin
0,A. Morin,Agua Grande,1876,2016,63%,France,3.75,,Sao Tome
1,A. Morin,Kpime,1676,2015,70%,France,2.75,,Togo
2,A. Morin,Atsane,1676,2015,70%,France,3.00,,Togo
3,A. Morin,Akata,1680,2015,70%,France,3.50,,Togo
4,A. Morin,Quilla,1704,2015,70%,France,3.50,,Peru
...,...,...,...,...,...,...,...,...,...
1790,Zotter,Peru,647,2011,70%,Austria,3.75,,Peru
1791,Zotter,Congo,749,2011,65%,Austria,3.00,Forastero,Congo
1792,Zotter,Kerala State,749,2011,65%,Austria,3.50,Forastero,India
1793,Zotter,Kerala State,781,2011,62%,Austria,3.25,,India


In [202]:
import re
for column in cc.columns:
    cc.columns = cc.columns.str.replace(r'\s*\n\s*', ' ', regex=True)
cc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1795 entries, 0 to 1794
Data columns (total 9 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Company (Maker-if known)          1795 non-null   object 
 1   Specific Bean Origin or Bar Name  1795 non-null   object 
 2   REF                               1795 non-null   int64  
 3   Review Date                       1795 non-null   int64  
 4   Cocoa Percent                     1795 non-null   object 
 5   Company Location                  1795 non-null   object 
 6   Rating                            1795 non-null   float64
 7   Bean Type                         1794 non-null   object 
 8   Broad Bean Origin                 1794 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 126.3+ KB


In [203]:
cc.rename(columns={
    'Company (Maker-if known)': 'Company',
    'Specific Bean Origin or Bar Name': 'BarName',
    'Review Date':'ReviewDate',
    'Cocoa Percent':'CocoaPercent',
    'Company Location':'CompanyLocation',
    'Bean Type':'BeanType',
    'Broad Bean Origin' : 'BroadBeanOrigin',
}, inplace=True)

In [204]:
cc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1795 entries, 0 to 1794
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Company          1795 non-null   object 
 1   BarName          1795 non-null   object 
 2   REF              1795 non-null   int64  
 3   ReviewDate       1795 non-null   int64  
 4   CocoaPercent     1795 non-null   object 
 5   CompanyLocation  1795 non-null   object 
 6   Rating           1795 non-null   float64
 7   BeanType         1794 non-null   object 
 8   BroadBeanOrigin  1794 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 126.3+ KB


In [205]:
cc.isnull().sum()

Company            0
BarName            0
REF                0
ReviewDate         0
CocoaPercent       0
CompanyLocation    0
Rating             0
BeanType           1
BroadBeanOrigin    1
dtype: int64

In [206]:
cc['CocoaPercent'] = cc['CocoaPercent'].str.replace('%', '').astype(float)
cc['CocoaPercent'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1795 entries, 0 to 1794
Series name: CocoaPercent
Non-Null Count  Dtype  
--------------  -----  
1795 non-null   float64
dtypes: float64(1)
memory usage: 14.2 KB


In [207]:
cc['BeanType'] = (cc['BeanType']
                 .str.strip()  # Menghapus leading/trailing spaces
                 .replace('', np.nan))  # Mengganti string kosong dengan NaN
cc['BeanType'] = cc['BeanType'].replace(np.nan, cc['BeanType'].mode()[0])

In [208]:
cc['ReviewDate'] = pd.to_datetime(cc['ReviewDate'].astype(str), format='%Y', errors='coerce')
cc['ReviewDate'] = cc['ReviewDate'].dt.year

In [209]:
cc.isnull().sum()
cc['BroadBeanOrigin'] = cc['BroadBeanOrigin'].fillna(cc['BroadBeanOrigin'].mode()[0])

In [210]:
cc_numerik = cc.select_dtypes(include=np.number)
cc_numerik.info()
cc_object = cc.select_dtypes(include='object')
cc_object.info()

# cc = pd.get_dummies(cc, columns=['Company','BarName','CompanyLocation','BeanType'], drop_first=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1795 entries, 0 to 1794
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   REF           1795 non-null   int64  
 1   ReviewDate    1795 non-null   int32  
 2   CocoaPercent  1795 non-null   float64
 3   Rating        1795 non-null   float64
dtypes: float64(2), int32(1), int64(1)
memory usage: 49.2 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1795 entries, 0 to 1794
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Company          1795 non-null   object
 1   BarName          1795 non-null   object
 2   CompanyLocation  1795 non-null   object
 3   BeanType         1795 non-null   object
 4   BroadBeanOrigin  1795 non-null   object
dtypes: object(5)
memory usage: 70.2+ KB


In [211]:
# le = LabelEncoder()
# categorical_features = ['BeanType', 'BroadBeanOrigin', 'CompanyLocation']
# for columns in categorical_features:
#     cc[columns] = le.fit_transform(cc[columns])

label_encoders = {}
categorical_features = ['BeanType', 'BarName', 'CompanyLocation']

for column in categorical_features:  
    label_encoders[column] = LabelEncoder()
    cc[column] = label_encoders[column].fit_transform(cc[column])
    
company_freq = cc['Company'].value_counts(normalize=True)
cc['Company'] = cc['Company'].map(company_freq)

In [212]:
cc.head()

,Company,BarName,REF,ReviewDate,CocoaPercent,CompanyLocation,Rating,BeanType,BroadBeanOrigin
0,0.012813,14,1876,2016,63.0,18,3.75,32,Sao Tome
1,0.012813,492,1676,2015,70.0,18,2.75,32,Togo
2,0.012813,67,1676,2015,70.0,18,3.00,32,Togo
3,0.012813,15,1680,2015,70.0,18,3.50,32,Togo
4,0.012813,809,1704,2015,70.0,18,3.50,32,Peru


# Modeling

In [213]:
X = cc.drop(columns='BroadBeanOrigin')
y = cc['BroadBeanOrigin']

In [214]:
# from imblearn.over_sampling import SMOTE


# smote = SMOTE(random_state=42)
# X_resampled, y_resampled = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [215]:
from imblearn.over_sampling import RandomOverSampler

# 1. Mari kita periksa distribusi kelas terlebih dahulu
print("Distribusi kelas original:")
print(pd.Series(y_train).value_counts())

# le = LabelEncoder()
# y_train_encoded = le.fit_transform(y_train)
# y_test_encoded = le.transform(y_test) 

# 2. Gunakan RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_train, y_train = ros.fit_resample(X_train, y_train)

# 3. Periksa distribusi kelas setelah oversampling
print("\nDistribusi kelas setelah oversampling:")
print(pd.Series(y_train).value_counts())

Distribusi kelas original:
BroadBeanOrigin
Venezuela                 176
Ecuador                   153
Peru                      142
Madagascar                113
Dominican Republic        110
                         ... 
Venezuela, Java             1
PNG, Vanuatu, Mad           1
Trinidad, Ecuador           1
Ghana, Panama, Ecuador      1
Ghana, Domin. Rep           1
Name: count, Length: 90, dtype: int64

Distribusi kelas setelah oversampling:
BroadBeanOrigin
Hawaii                    176
Sri Lanka                 176
South America, Africa     176
Ghana, Panama, Ecuador    176
St. Lucia                 176
                         ... 
Gabon                     176
Philippines               176
Central and S. America    176
Nicaragua                 176
Ghana, Domin. Rep         176
Name: count, Length: 90, dtype: int64


In [216]:
# LRclassifier = LogisticRegression(solver='saga', random_state=42)
# LRclassifier.fit(X_train, y_train)

In [217]:
def f1_score(y_true, y_pred):
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))

    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)

    f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
    return f1

In [218]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 200),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'random_state': 42
    }
    
    clf = RandomForestClassifier(**params)
    
    # Gunakan cross validation
    scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='f1_macro')
    return scores.mean()

# Create study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

print('Best parameters:', study.best_params)
print('Best f1-macro score:', study.best_value)

# Evaluasi final model
best_rf = RandomForestClassifier(**study.best_params)
best_rf.fit(X_train, y_train)
y_pred = best_rf.predict(X_test)

from sklearn.metrics import classification_report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

[I 2025-05-17 15:21:28,707] A new study created in memory with name: no-name-02650663-3357-4b15-b9e8-a7d1ae29bbed
[I 2025-05-17 15:21:40,023] Trial 0 finished with value: 0.9621365285010457 and parameters: {'n_estimators': 200, 'max_depth': 18, 'min_samples_split': 10}. Best is trial 0 with value: 0.9621365285010457.
[I 2025-05-17 15:21:41,862] Trial 1 finished with value: 0.5711192074717 and parameters: {'n_estimators': 48, 'max_depth': 6, 'min_samples_split': 9}. Best is trial 0 with value: 0.9621365285010457.
[I 2025-05-17 15:21:46,771] Trial 2 finished with value: 0.45757793988620454 and parameters: {'n_estimators': 165, 'max_depth': 4, 'min_samples_split': 17}. Best is trial 0 with value: 0.9621365285010457.
[I 2025-05-17 15:21:48,484] Trial 3 finished with value: 0.8235469889525892 and parameters: {'n_estimators': 34, 'max_depth': 11, 'min_samples_split': 15}. Best is trial 0 with value: 0.9621365285010457.
[I 2025-05-17 15:21:55,535] Trial 4 finished with value: 0.80771059951960

Best parameters: {'n_estimators': 99, 'max_depth': 20, 'min_samples_split': 2}
Best f1-macro score: 0.974886298889675

Classification Report:
                               precision    recall  f1-score   support

    Africa, Carribean, C. Am.       0.00      0.00      0.00         1
                    Australia       0.00      0.00      0.00         1
                       Belize       0.35      0.67      0.46         9
                      Bolivia       0.60      0.46      0.52        13
                       Brazil       0.64      0.58      0.61        12
                     Cameroon       0.00      0.00      0.00         1
                    Carribean       0.33      1.00      0.50         1
       Central and S. America       0.00      0.00      0.00         1
                     Colombia       0.36      0.67      0.47         6
                        Congo       0.00      0.00      0.00         3
                   Costa Rica       0.25      0.22      0.24         9
     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.